# PHYS366: Statistical Methods in Astrophysics


# Lesson 5: Inference in Practice: Coping With ~~Complications~~ Reality

### Goals for this session:

* Use hierachical models to encode physical relationships among sources
* Design models that account for the imperfections of data collection
* Cope with situations where we don't know how to build a model

#### Related reading:
* Bits of Gelman chapters 3, 5 and 7
* Hogg et al., [Data analysis recipes: Fitting a model to data](http://arxiv.org/abs/1008.4686)

## About this session
#### We started this course by looking at astronomical data and talking about some of its nasty features. Now that you understand the inferential tools we have at our disposal and have tested them out on relatively simple problems, it's time to get back to the dirty business of drawing conclusions from real data, warts and all.

## Setting the stage

#### Below are three real astronomical data sets, all presenting different aspects of the "line-fit" problem we've seen before.

<table width="100%">
    <tr>
        <td><img src="../notes/CopingExample1.png" width=320></td>
        <td><img src="../notes/CopingExample2.png" width=320></td>
        <td><img src="../notes/CopingExample3.png" width=320></td>
    </tr>
</table>

### Question: What features can you identify in these data sets that don't fit into the simple model we've used previously?

* sweet, sweet upper bounds
* uncertainties in both axes
* skewed confidence intervals and measurement positions
* it's not mock data
* what are those lines and what are they doing on my data plot
* Adam Riess and Nobel laureates
* neutrino flavours how do they work

Common answers:
<ol start="0">
<li>shape of sampling distribution unknown [i.e. it's not mock data]
</li>
<li> intrinsic scatter [i.e. it's not mock data]</li>
<li> interlopers/outliers [i.e. it's not mock data]</li>
<li> limits without centres (statistical censorship)</li>
<li> $x$ errors and non-zero covariance off-diagonals</li>
<li> selection effects, truncated data [i.e. it's not mock data]</li></ol>

### Let's see how we can include these features in a model. To the whiteboard!

We're familiar with a generative model: parameters $a$, $b$, ..., plate containing fixed data $x_i$, $\sigma_i$, ... conditional output $y_i$ ...

$$P(a,b|x,y) = \prod_i P(y_i|x_i,a,b)\cdot P(a,b)$$

We usually think of $P(\dots)$ as Gaussian, but of course none of them have to be. This resolves point 0 and allows us to start with point 1 because I can't be bothered to do raw HTML again.

1. **Intrinsic scatter:** add another parameter (outside the plate) $\tau$ along with $a,b$, for which we need a prior, just like any other model parameter. We have a 'true' value of $y_\text{true}$ which is scattered randomly with width $\tau$, and then we have measurement uncertainty $\sigma$ on top of this to give the measured $y$.
$$P(a,b,\tau|x,y) = \prod_i P(y_i|y_{i,\text{true}},\sigma_i)P(y_{i,\text{true}}|x_i,a,b,\tau)\cdot P(a,b,\tau)$$
    If we find we have some idea of $y_\text{true}$, we could integrate it out, but sometimes that's not actually efficient.
    
    A subtlety: we have a prior for $y_\text{true}$ ... we've got a (particularly) hierarchical model on our hands.
2. **Outliers:** what are they, really? They're not even really well-defined. They're entirely designated by the fact that they don't fit our model. Well, maybe we should alter our model! Maybe we should combine multiple models together! So, modelling the outliers on their own (so that they aren't outliers anymore) should be sufficient.
3. Oh god we're running out of time [it's 16:17] so let's adjourn for today.
    
    <hr />
    
    Okay, it's Thursday. Revisiting outliers briefly, recall that for outliers, there isn't a single 'right' way to deal with this, but the Bayesian way is to adjust your model, perhaps as a combination of two populations. Either we turn $\tau$ into a more complicated prior, or we add on a plate to our PGM to go between the different populations. We have discrete group labels (for each distribution $j$), and we have the population scatter determined by $\tau_j$ rather than a single $\tau$.
    $$P(a,b,\tau|x,y) = \prod_i P(y_i|y_{i,\text{true}},\sigma_i)P(y_{i,\text{true}}|x_i,a,b,\tau,G)\cdot P(a,b,\tau,G)$$
    Now, back to **limits:** what is missing from our knowledge that would lead to a downwards arrow? [Well, mixing log and linear scales, for a start ... ex. if your posterior is on the log of the quantity and you end up with a lower bound < 0 ...] How would priors and likelihoods give rise to such limits?
    * We're missing information on the likelihood, which results in failure to fully constrain the result.
    * Extreme priors would give rise to such limits, if we had some kind of step function prior, causing us to utterly constrain on one end and utterly be unable to constrain on the other end. (Wouldn't this be technically improper without an arbitrary cutoff? Well, yes. There's a severe prior dependence here.)
    
    We make no changes, because the one-sided limit accurately reflects our beliefs (either that or you're believing badly).
4. **Non-trivial covariance**: how does it work? Well, first off, $x$ isn't a fixed point anymore—it's drawn from a distribution of finite width, with a true value buffered from our $y_\text{true}$. The true value is a circle, and feeds into $y_\text{true}$ as it did before. So now we have constants $x,y$ (oh, throw them into a vector), connected to a covariance matrix $\Sigma$. Furthermore, $x_\text{true}$ still has to come from somewhere (as does $G$, actually!).
    $$P(a,b,\tau,x_\text{true},y_\text{true},G|x,y,\Sigma) = \prod_i P((x_i,y_i)|x_{i,\text{true}},y_{i,\text{true}},\Sigma_i)P(y_{i,\text{true}}|x_{i,\text{true}},a,b,\tau,G)P(x_{i,\text{true}})\cdot P(a,b,\tau,G)$$
    [we're absorbing where $x_\text{true}$ and $G$ come from into our grand set of assumptions $H$] ... and yes, we're just going to say we know $\Sigma$ sufficiently well.    
5. **Selection effects:** This is a doozy. We have terrible, terrible bias towards higher-flux objects, and our mean relation will be distorted as a result of this.
    
    Directly modelling the selection bias (as a fixed point, if we understand it well enough) comes to mind. However, suppose we know what our threshold is—and note that since the threshold affects what we're observing [unless it's a physical hard cutoff—I dunno, halo mass vs CO luminosity?—but this probably is a model parameter, not a selection bias effect], it seems that this will be only a threshold on the observed values, not the 'true' values.
    
    We must adjust our likelihood function $P(x,y|\cdots)$, in fact, so as to cut it down to zero beyond the threshold. The likelihood function only applies to points in our observations, so this isn't catastrophic—we will want to throw in another label $I$ to indicate whether the data point is in our observed sample (erm, that plus our actual understanding of the threshold).
    
    Go back to the PGM. We're just missing a fair sample of the population beyond the threshold, which means we are missing the true size of the population, and thus the true weight of each point. We're trying to describe population statistics here, or at least that's what we want to do most of the time. Go back to thinking about the forward/generative model—we want a good description of all possible samples.
    
    So we must extend our label $i$ to everything. $I$ for each sample is fully conditional based on $\vec{x}_\text{true}$, but since the blasted thing isn't exactly drawn from a distribution, let's use a dot. (Or just a double-circle of radius zero.)
    
    $N_\text{tot}$ is modelled, so we use a single circle outside the plate. The number of actual samples determines this to some extent, and that has some randomness to it, so let's throw a double circle around that. [To be perfectly sure of whether we deal with dots or circles, you must first re-create the Universe. Many, many times.] Of course $N_\text{tot}$ will connect to our rectangle, and $N_\text{det}$ will do just connect to $N_\text{tot}$.
    
    $$P(a,b,\tau,x_\text{true},y_\text{true},G,N_\text{tot}|x,y,\Sigma) = \prod_i P((x_i,y_i)|x_{i,\text{true}},y_{i,\text{true}},\Sigma_i,I)P(y_{i,\text{true}}|x_{i,\text{true}},a,b,\tau,G)P(x_{i,\text{true}})\cdot P(a,b,\tau,G)P(N_\text{tot})P(N_\text{det}|N_\text{tot}){N_\text{tot}\choose N_\text{det}}$$
    
    ($N_\text{det}$ is implicitly thrown into our data vectors.) Depending on how we model $N_\text{tot}$, of course (for clusters we could have some dependence on $x_\text{true}$), we're going to have additional nodes and edges. We do indeed need to account for all the different ways in which we could have lost $N_\text{tot}-N_\text{det}$ samples, which is the reason for our combinatoric factor.
    
    Note that we have had to add on at least three nuisance parameters (true data vectors plus $G$) to *fit a linear model*. (We're arguably at least somewhat interested in how complete our model is.) In practice, we don't want to navigate a 7-dimensional parameter space, and there are ways to make this sort of compensation much more tractable.

## [Detour: "less probabilistic" methods](../notes/bootjackabc.ipynb)

Quick primer on Malmquist vs Eddington (Malmquist–Eddington bias?):
* By Malmquist bias, we typically mean that we are biased towards brighter sources because they're brighter and actually even observable, which affects our inference of the model parameters.
* By Eddington bias, we typically mean that we're interested in $N_\text{tot}$ and because we're biased towards brighter sources, we bias the slope badly. [Or something.]